# Ethereum SQL DB - Analysis and Export

The purpose of this file is to get an overview of the data currently stored in the data base and to make data exports.
The data stored in the database come from all scrapers starting with 01

Table of content:
<br> 1. Connect to the data base
<br> 2. Analysis
<br> 3. Data exports

## 1. Connect to the data base

In [3]:
# import libraries
import psycopg2
import ast


host = "localhost"
dbname = "postgres"
user = "postgres"
password = "" # load password from environment

# function to connect to db
def connect_database(host, dbname, user, password):

    """
    Connects to the existing ethereum_data database, this DB is created in SQL commandline and can be viewd under pg admin
    Arguments:
        - host (String): location of db; localhost or IP address
        - dbname (String): has to be created in pg admin
        - user (String): default is postgres -> or create new user in pg admin
        - password (String): password of user

    Returns two objects a cursor and a connection to the db 

    """
    conn = psycopg2.connect("host="+host+" dbname="+dbname+" user="+user+" password="+password)
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    print("has successfully connceted to db")
    

    return cur, conn



def disconnect_database(conn):

    """
    this function disconnect from the database"""

    # close connections to the database
    conn.close()

    print("has successfully disconnceted from db")




In [110]:
cur, conn = connect_database(host, dbname, user, password)

has successfully connceted to db


## 2. Analysis 

In [4]:
# send queries to DB
def call_query(query):

    import psycopg2

    """
    This function can be used to send queries to the db.
    It automatically connects to the db, sends the query, and returns the result

    Argument:
    query (string): SQL command with 3x quotes

    Returns the result from the db as a list with fetchall()
    """

    # connects to the db
    cur, conn = connect_database(host, dbname, user, password)

    # sends query to db
    cur.execute(query)
    
    # stores results
    results = cur.fetchall()
    print("has successfully received results")

    # closes connection to the db
    disconnect_database(conn)

    # returns results
    return results

In [22]:
# write your query here
#query = """SELECT d_name, d_contract_list FROM dapp_data """
query = """SELECT * FROM contract_data"""

In [23]:
res = call_query(query)

has successfully connceted to db
has successfully received results
has successfully disconnceted from db


In [25]:
res

[]

## 3. Data exports

### Export dapp_data table
Exports the whole dapp_data table and dumps it into one csv file

In [11]:
import csv

data = call_query("""SELECT * FROM dapp_data""")

with open('dump_dapps_data.csv','w', encoding='utf-8',newline='') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['d_name','d_teaser', 
                            'd_platform', 
                            'd_cat',
                            'd_link',
                            'd_descr',
                            'd_status',
                            'd_author',
                            'd_license',
                            'd_updated',
                            'd_submitted',
                            'd_website',
                            'd_profile_str',
                            'd_push',
                            'd_pull',
                            'd_users_d',
                            'd_users_w',
                            'd_users_m',
                            'd_txn_1',
                            'd_txn_7',
                            'd_txn_30',
                            'd_eth_1',
                            'd_eth_7',
                            'd_eth_30',
                            'd_contract_list',
                            'd_review_list',
                            'd_views',
                            'd_clicks',
                            'd_ctr',
                            'd_rel_dapp_list',
                            'd_social_list',
                            'd_reaction_pos',
                            'd_reaction_neu',
                            'd_reaction_neg',
                            'd_tag_list',
                            'd_metamask_recom',
                            'd_ponzi_warning',
                            'added'])
    for row in data:
        csv_out.writerow(row)

print("file has been saved")

has successfully connceted to db
has successfully received results
has successfully disconnceted from db
file has been saved


### Export contract_data table
Exports the whole contract_data table and dumps it into one csv file

In [28]:
import csv

data = call_query("""SELECT * FROM contract_data""")

with open('dump_contract_data.csv','w', encoding='utf-8',newline='') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['c_address',
                            'c_name', 
                            'c_creator',
                            'c_token_traker',
                            'c_compiler',
                            'c_verified',
                            'c_creation',
                            'c_source_code'])
    for row in data:
        csv_out.writerow(row)

print("file has been saved")

has successfully connceted to db
has successfully received results
has successfully disconnceted from db
file has been saved


### Dapps and Contract export
exports a csv file containing all dapp names with all their contracts in a seperate row

In [13]:
# export dapps with contract list 

import csv

data = call_query("""SELECT d_name, c_address FROM dapp_contract_link """)

with open('dapps_contract_link_test.csv','w', encoding='utf-8',newline='') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['d_name','c_address'])
    for row in data:
        csv_out.writerow(row)

print("file has been saved")

has successfully connceted to db
has successfully received results
has successfully disconnceted from db
file has been saved


### All dApps contract and high level info export
this routines selects all dapps (including some high level info s.a. category) and joins these dapps with the contract link table and returns all contracts associated these dApps

In [15]:
query = """
        WITH dapps AS (
            SELECT 
                d.d_name, 
                d.d_submitted, 
                d.d_cat, 
                d.d_status, 
                d.d_tag_list, 
                d.d_metamask_recom
            FROM dapp_data d
            WHERE 
                  LOWER(d_status) LIKE '%live%' OR  
                  LOWER(d_status) LIKE '%abandoned%' OR
                  LOWER(d_status) LIKE '%beta%'
                        ) 
            SELECT 
                dapps.d_name, 
                dapps.d_cat, 
                dapps.d_status, 
                dapps.d_tag_list,  
                dapp_contract_link.c_address
            FROM dapps 
            JOIN dapp_contract_link 
            ON dapps.d_name = dapp_contract_link.d_name
        """

In [19]:
# export dapps with contract list 

import csv

data = call_query(query)

with open('dapps_contract_link.csv','w', encoding='utf-8',newline='') as out:
    csv_out=csv.writer(out, delimiter=";")
    csv_out.writerow(['d_name','d_cat','d_status' ,'d_tag_list', 'c_address'])
    for row in data:
        csv_out.writerow(row)

print("file has been saved")

has successfully connceted to db
has successfully received results
has successfully disconnceted from db
file has been saved


### Defi dApps Contract Export
this routines filters all dapps by looking for defi dapps, joins these dapps with the contract link table and returns alls contracts associated with defi dapps

In [133]:
# define sql query
    # problem: false positives and false negatives due to matching approach. Manual matching is required

query = """WITH defi_dapps AS (
    SELECT d_name, d_submitted 
    FROM dapp_data
    WHERE  LOWER(d_tag_list) LIKE '%swap%' OR
            LOWER(d_tag_list) LIKE '%dex%' OR 
            LOWER(d_tag_list) LIKE '%liquid%' OR 
            LOWER(d_tag_list) LIKE '%defi%' OR 
            LOWER(d_tag_list) LIKE '%liquidity%' OR 
            LOWER(d_tag_list) LIKE '%staking%' OR 
            LOWER(d_tag_list) LIKE '%swapping%' OR 
            LOWER(d_tag_list) LIKE '%lending%' OR 
            LOWER(d_tag_list) LIKE '%loans%' OR
            LOWER(d_tag_list) LIKE '%trading%'
            ) 

    SELECT defi_dapps.d_name, defi_dapps.d_submitted, dapp_contract_link.c_address
    FROM defi_dapps
    JOIN dapp_contract_link
    ON defi_dapps.d_name = dapp_contract_link.d_name"""

In [57]:
# export dapps with contract list 

import csv

data = call_query(query)

with open('defi_dapps_contract_link.csv','w', encoding='utf-8',newline='',delimiter=";") as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['d_name','submitted' ,'c_address'])
    for row in data:
        csv_out.writerow(row)

print("file has been saved")

has successfully connceted to db
has successfully received results
has successfully disconnceted from db
file has been saved
